In [3]:
import numpy as np
import yaml
from astropy.table import Table
from astropy.io import fits
import desispec.maskbits as masks
import desimodel.io

# YAML target description

In [4]:
stream = open('../../desidata/inputs/targets.yaml', 'r')
_alldefs = yaml.load(stream)
stream.close()
attr = []
for line in _alldefs["targetmask"]:
    attr.append(line[:3])
_targetbitdefs = dict([("targetmask", attr)])
print _targetbitdefs

{'targetmask': [['QSOLYA', 0, 'Lyman alpha QSO'], ['QSO', 1, 'Tracer QSO'], ['LRG', 2, 'Luminous Red Galaxy'], ['ELG', 3, 'Emission Line Galaxy'], ['STAR', 4, 'Standard Star'], ['SKY', 5, 'Sky'], ['BGS', 6, 'Bright Galaxy']]}


In [5]:
#Mock set of random objects

In [6]:
targetmask = masks.BitMask("targetmask", _targetbitdefs)
n_tiles = 10
target_per_tile = 25000
n_fibers = 5000
total_n_targets = target_per_tile * n_tiles / 2
random_ID_list =  np.arange(total_n_targets, dtype='int64')
random_ra_list = np.random.random(total_n_targets)
random_dec_list = np.random.random(total_n_targets)
random_type_A_list = np.int_(np.random.random(total_n_targets)*7)
random_type_B_list = np.int_(np.random.random(total_n_targets)*7)
print np.size(np.where(random_type_A_list == random_type_B_list))

18231


# DESI targets

In [7]:
def write_target_fits(id_list, ra_list, dec_list, type_list, filename):
    t = Table([id_list, ra_list, dec_list, type_list], names=('ID', 'RA', 'DEC', 'TYPE'), meta={'name': 'target table'})
    t.write(filename, format='fits', overwrite=True)

    
for i_tile in range(n_tiles):
    #choose randomly targets on different tiles
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    tile_id = np.random.choice(random_ID_list, replace=False,size=target_per_tile)
    tile_ra = random_ra_list[tile_id]
    tile_dec = random_dec_list[tile_id]
    tile_type_A = random_type_A_list[tile_id]
    tile_type_B = random_type_B_list[tile_id] 
    tile_type = np.ones(target_per_tile, dtype='int64')
    
    #create mock types
    for i in range(target_per_tile):
        tile_type[i]  = targetmask.mask(random_type_A_list[i]) | targetmask.mask(random_type_B_list[i])
        
    #write and read
    write_target_fits(tile_id, tile_ra, tile_dec, tile_type, filename)
    t = Table.read(filename, format='fits')

# DESI observations

Tracks what objects have been targetted.
* Object-id (int)
* ObservedFlag (int) Yes/No (1/0) flag.

In [8]:
def write_observations_fits(id_list, observed_list, filename):
    t = Table([id_list, observed_list], names=('ID', 'OBSFLAG'), meta={'name': 'observations table'})
    t.write(filename, format='fits', overwrite=True)

for i_tile in range(n_tiles):
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    t = Table.read(filename, format='fits')
    
    object_id = t['ID']
    observed_id = np.random.choice(np.arange(np.size(object_id)), replace=False,size=n_fibers)
    observed_flag = np.zeros(np.size(object_id), dtype=int)
    observed_flag[observed_id] = 1
    print sum(observed_flag), np.size(observed_id), observed_id
    
    filename = 'tmp/observations_%06d.fits'%(i_tile)
    write_observations_fits(object_id, observed_flag, filename)
    

5000 5000 [16421 21824  5933 ..., 22118 23745 16436]
5000 5000 [ 3624  9265  4258 ...,  6416 20695  7435]
5000 5000 [  330 16472 14391 ..., 17384  2021 14548]
5000 5000 [16998 16060 12086 ..., 19569 12641 19776]
5000 5000 [ 8100 14422 10695 ...,  8353  4915  7988]
5000 5000 [16442 20157  9850 ...,  7653  8153 24228]
5000 5000 [ 6584 16839 24967 ..., 11895  7662   781]
5000 5000 [22190 21057  5414 ..., 14571 19021 23806]
5000 5000 [2900 9044 2871 ..., 4691 6893 6162]
5000 5000 [14028 15974  4148 ...,  8127 19260  8473]


# DESI results

Tracks the results of processed observed tiles

* Object-id (int)
* DesiType (int64) Mask
* z_spec (float)

In [9]:
def write_results_fits(id_list, type_list, redshift_list, filename):
    t = Table([id_list, type_list, redshift_list], names=('ID', 'TYPE', 'REDSHIFT_LIST'), meta={'name': 'results table'})
    t.write(filename, format='fits', overwrite=True)

#generates mock results for each tile
for i_tile in range(n_tiles):
    target_file = 'tmp/targets_%06d.fits'%(i_tile)
    observations_file = 'tmp/observations_%06d.fits'%(i_tile)
    targets = Table.read(target_file, format='fits')
    observations = Table.read(observations_file, format='fits')
    
    
    targets_id = targets['ID']
    observations_id = observations['ID']
    n_targets = np.size(targets_id)
    
    assert np.size(targets_id)==np.size(observations_id)
    
    z_list = np.zeros(n_targets)
    type_list  = targets['TYPE'].copy()
    for i in range(n_targets):
        z_list[i] = -1.0
        type_list[i] = targets['TYPE'][i]
        if(observations['OBSFLAG'][i]==1):
            z_list[i] = np.random.random()
            type_bit_offset = np.int_(np.random.random()*7)
            type_list[i] = targetmask.mask(type_bit_offset)
    
    filename = 'tmp/results_%06d.fits'%(i_tile)
    write_results_fits(targets_id, type_list, z_list, filename)

# Merged Target List

Uses all the targets/observations/results files to produce a single file to be fed into fiber assignment

# Fiber assignment

In [11]:
#load fibers
fibers = desimodel.io.load_fiberpos()
fiberid = fibers['FIBER']
